Author: Dhruval PB (https://github.com/Dhruval360/CNNs-from-scratch.git)

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow # Used only for loading the dataset

# Loading the dataset

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tensorflow.keras.datasets.mnist.load_data()

num = 10
images = train_images[:num]
labels = train_labels[:num]
num_row = 2
num_col = 5

fig, axes = plt.subplots(num_row, num_col, figsize=(2*num_col,2*num_row))
for i in range(num):
    ax = axes[i//num_col, i%num_col]
    ax.imshow(images[i], cmap='gray')
    ax.set_title('Label: {}'.format(labels[i]))
plt.tight_layout()
plt.show()

print("Train images shape: ", train_images.shape)
print("Testing images shape: ", test_images.shape)

# Convolution layer
* This layer is used for feature extraction
* Multiple filters are applied on the input to obtain different features
* The size of the output after applying convolution is given by:

$$O = \frac{I - F + 2P}{S} + 1$$

        Where,
        O = Output size
        I = Input size
        F = Filter size
        S = Stride
        P = Padding 

In [ ]:
class Conv2D:
    def __init__(self, input_shape, num_filters, filter_size, stride, padding = 0):
        self.output_shape = (int(input_shape[0] - filter_size[0] + 2*padding)//stride + 1, int(input_shape[1] - filter_size[1] + 2*padding)//stride + 1, num_filters)

        # Hyperparameters
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.stride = stride
        self.padding = padding
                
        # Trainable parameter (weights of the filter)
        self.conv_filter = np.random.normal(size = (num_filters, filter_size[0] , filter_size[1])) # Initialising the filters with random normalised floats
    
    def image_region(self, image): # A Generator function to extract patches of the image
        height, width = image.shape
        for j in range(0, height - self.filter_size[0] + 1, self.stride):
            for k in range(0, width - self.filter_size[1] + 1, self.stride):
                image_patch = image[j : (j + self.filter_size[0]), k : (k + self.filter_size[1])]
                yield image_patch
    
    def pad(self, input):
        output_shape = (input.shape[0] + 2*self.padding, input.shape[1] + 2*self.padding)
        result = np.zeros(output_shape) 
        result[self.padding : (input.shape[0] + self.padding), self.padding : (input.shape[1] + self.padding)] = input
        return result

    def forward_prop(self, image): # Defining the convolution operation
        self.image = self.pad(image)
        height, width = self.image.shape
        self.conv_out = np.zeros(((height - self.filter_size[0])//self.stride + 1, (width - self.filter_size[1])//self.stride + 1, self.num_filters)) 

        image_patch = self.image_region(self.image)
        for i in range(self.conv_out.shape[0]):
            for j in range(self.conv_out.shape[1]):
                self.conv_out[i,j] = np.sum(next(image_patch) * self.conv_filter, axis = (1,2))
        return self.conv_out
    
    def back_prop(self, dL_dout, learning_rate): # The differential dL/dout is obtained from the max pooling layer
        dL_dF_params = np.zeros(self.conv_filter.shape)
        
        image_patch = self.image_region(self.image)
        for i in range(self.conv_out.shape[0]):
            for j in range(self.conv_out.shape[1]):
                patch = next(image_patch)
                for k in range(self.num_filters):
                    dL_dF_params[k] += patch * dL_dout[i,j,k]
        
        # Update the filter weights
        self.conv_filter -= learning_rate * dL_dF_params
        return dL_dF_params

# Maxpooling layer
* The main function of this layer is to reduce the dimension of the input
* It forwards only the most prominent parts of the input to the next layer (pixels with the highest value)
* As some data is lost in this process, it is always better to use a maxpooling layer with a small filter size
* This layer does not have any trainable parameters
* The size of the output after applying maxpooling is given by:

$$O = \frac{I - F + 2P}{S} + 1$$

        Where,
        O = Output size
        I = Input size
        F = Filter size
        S = Stride
        P = Padding 

In [ ]:
class MaxPool:
    def __init__(self, input_shape, filter_size, stride, padding = 0):
        self.output_shape = ((input_shape[0] - filter_size[0] + 2*padding)//stride + 1, (input_shape[1] - filter_size[1] + 2*padding)//stride + 1, input_shape[2])

        # Hyperparameters
        self.filter_size = filter_size
        self.stride = stride
        self.padding = padding
    
    def image_region(self, image): # A Generator function to extract patches of the image
        height, width = image.shape[0], image.shape[1]
        for j in range(0, height - self.filter_size[0] + 1, self.stride):
            for k in range(0, width - self.filter_size[1] + 1, self.stride):
                image_patch = image[j : (j + self.filter_size[0]), k : (k + self.filter_size[1])]
                yield image_patch

    def pad(self, input): # This pad function has to pad all the layers of the input
        output_shape = (input.shape[0] + 2*self.padding, input.shape[1] + 2*self.padding, input.shape[2])
        result = np.zeros(output_shape) 
        for i in range(input.shape[2]):
            result[self.padding : (input.shape[0] + self.padding), self.padding : (input.shape[1] + self.padding), i] = input[:,:,i]
        return result

    def forward_prop(self, image): # Defining the MaxPooling operation
        self.image = self.pad(image)        
        height, width, num_filters = self.image.shape
        
        self.output = np.zeros(((height - self.filter_size[0])//self.stride + 1, (width - self.filter_size[1])//self.stride + 1, num_filters)) 

        image_patch = self.image_region(self.image)
        for i in range(self.output.shape[0]):
            for j in range(self.output.shape[1]):
                self.output[i,j] = np.amax(next(image_patch), axis = (0,1))
            
        return self.output
    

    def back_prop(self, dL_dout, learning_rate): # This layer does not have any trainable parameters (no weights or biases)
        dL_dmax_pool = np.zeros(self.image.shape)
        image_patch = self.image_region(self.image)
        for i in range(self.output.shape[0]):
            for j in range(self.output.shape[1]):
                patch = next(image_patch)
                height, width, num_filters = patch.shape
                maximum_val = np.amax(patch, axis = (0,1))
                
                for i1 in range(height):
                    for j1 in range(width):
                        for k1 in range(num_filters):
                            if patch[i1, j1, k1] == maximum_val[k1]:
                                dL_dmax_pool[i*self.filter_size[0] + i1, j*self.filter_size[1] + j1, k1] = dL_dout[i, j, k1]
        return dL_dmax_pool

# Softmax layer
* The softmax function is a function that turns a vector of K real values into a vector of K real values that sum to 1.
* The input values can be positive, negative, zero, or greater than one, but the softmax transforms them into values between 0 and 1, so that they can be interpreted as probabilities. 
* If one of the inputs is small or negative, the softmax turns it into a small probability, and if an input is large, then it turns it into a large probability, but it will always remain between 0 and 1.
* The softmax function is sometimes called the softargmax function, or multi-class logistic regression. This is because the softmax is a generalization of logistic regression that can be used for multi-class classification, and its formula is very similar to the sigmoid function which is used for logistic regression. 
* The softmax function can be used in a classifier only when the classes are mutually exclusive.
* Many multi-layer neural networks end in a penultimate layer which outputs real-valued scores that are not conveniently scaled and which may be difficult to work with. Here the softmax is very useful because it converts the scores to a normalized probability distribution, which can be displayed to a user or used as input to other systems. For this reason it is usual to append a softmax function as the final layer of the neural network.
* The softmax function is given by
$$\sigma(\overrightarrow{Z})_{i} = \frac{e^{Z_{i}}}{\sum\limits_{j=1}^{K}e^{Z_{j}}}$$

Where,

$\overrightarrow{Z}$ = The input vector to the softmax function, made up of (z0, ... zK)

$Z_{i}$ = All the zi values are the elements of the input vector to the softmax function, and they can take any real value, positive, zero or negative. For example a neural network could have output a vector such as (-0.62, 8.12, 2.53), which is not a valid probability distribution, hence why the softmax would be necessary.
	
$e^{Z_{i}}$ = The standard exponential function is applied to each element of the input vector. This gives a positive value above 0, which will be very small if the input was negative, and very large if the input was large. However, it is still not fixed in the range (0, 1) which is what is required of a probability.
	
$\sum\limits_{j=1}^{K}e^{Z_{j}}$ = The term on the bottom of the formula is the normalization term. It ensures that all the output values of the function will sum to 1 and each be in the range (0, 1), thus constituting a valid probability distribution.
	
K = The number of classes in the multi-class classifier.

In [ ]:
class Softmax:
    def __init__(self, input_node, softmax_node):
        input_node = int(input_node)
        self.weight = np.random.randn(input_node, softmax_node) / input_node
        self.bias = np.zeros(softmax_node)
    
    def forward_prop(self, image):
        self.orig_im_shape = image.shape # Used in backprop
        image_modified = image.flatten()
        self.modified_input = image_modified # To be used in backprop
        output_val = np.dot(image_modified, self.weight) + self.bias
        self.out = output_val
        exp_out = np.exp(output_val)
        return exp_out/np.sum(exp_out, axis = 0) # Probability output
    
    def back_prop(self, dL_dout, learning_rate):
        for i, grad in enumerate(dL_dout):
            if grad == 0:
                continue
            
            transformation_eq = np.exp(self.out)
            S_total = np.sum(transformation_eq)

            # Gradients with respect to out (z)
            dy_dz = -transformation_eq[i] * transformation_eq / (S_total** 2)
            dy_dz[i] = transformation_eq[i]*(S_total - transformation_eq[i]) / (S_total**2)

            # Gradients of totals against weights/biases/input
            dz_dw = self.modified_input
            dz_db = 1
            dz_d_inp = self.weight

            # Gradients of loss against totals
            dL_dz = grad * dy_dz

            # Gradients of loss against weights/biases/input
            dL_dw = dz_dw[np.newaxis].T @ dL_dz[np.newaxis]
            dL_db = dL_dz * dz_db
            dL_d_inp = dz_d_inp @ dL_dz

        # Update weights and biases
        self.weight -= learning_rate * dL_dw
        self.bias -= learning_rate * dL_db
        return dL_d_inp.reshape(self.orig_im_shape)

# Example outputs

In [ ]:
index = 7
image_in = images[index]
label = labels[index]
num_col = 10
num_row = 2
num = 10

layer1 = Conv2D(image_in.shape, 10, (3,3), stride = 2, padding = 0) # A convolutional layer with 10 filters, each of size 3x3
layer2 = MaxPool(layer1.output_shape, (3,3), stride = 3, padding = 0)   # A Max pooling layer with filter size 3x3
layer3 = Softmax(np.prod(layer2.output_shape, axis=None), 10) # Softmax layer

t1 = time.time()
output1 = layer1.forward_prop(image_in) # Applying the Convolution
t2 = time.time()
print("Time taken for Convolution layer = ", t2-t1, "s")

t1 = time.time()
output2 = layer2.forward_prop(output1) # Applying Max Pooling
t2 = time.time()
print("Time taken for MaxPooling layer = ", t2-t1, "s")

t1 = time.time()
output3 = layer3.forward_prop(output2) # Applying Softmax
t2 = time.time()
print("Time taken for Softmax layer = ", t2-t1, "s")

fig, axes = plt.subplots(num_row, num_col, figsize=(2*num_col,2*num_row))
for i in range(num):
    ax = axes[0, i]
    ax.imshow(output1[:,:,i], cmap='gray')
    ax.set_title('Label: {}'.format(labels[index]))

    ax = axes[1, i]
    ax.imshow(output2[:,:,i], cmap='gray')
    ax.set_title('Label: {}'.format(labels[index]))
    
axes[0, 0].set_ylabel("Convolution output", rotation=90, size='large')
axes[1, 0].set_ylabel("MaxPooling output", rotation=90, size='large')


plt.tight_layout()
plt.show()

print("Softmax output: ", output3)
print("\nPrediction: ", np.argmax(output3))

# Defining the model and writing wrapper functions

In [ ]:
# Model
conv = Conv2D((train_images.shape[1], train_images.shape[2]), 8, (2,2), stride = 2, padding = 1)
pool = MaxPool(conv.output_shape, (2,2), stride = 2, padding = 1)
softmax = Softmax(np.prod(pool.output_shape, axis=None), 10)

model = [conv, pool, softmax]

In [ ]:
# A wrapper function that does only forward propagation 
def cnn_forward_prop(image, label, layers):
    out_p = (image/255) - 0.5
    for i in layers:
      out_p = i.forward_prop(out_p)

    # Calculating cross-entropy loss and accuracy
    cross_ent_loss = -np.log(out_p[label])
    accuracy_eval = 1 if np.argmax(out_p) == label else 0

    return out_p, cross_ent_loss, accuracy_eval

# A wrapper function that does both forward and backward propagation (training)
def training_cnn(image, labl, layers, learn_rate = 0.005):
    # Forward Prop
    out, loss, acc = cnn_forward_prop(image, label, layers)

    # Calculate initial gradient
    gradient = np.zeros(10)
    gradient[label] = -1/out[label]

    # Backprop
    grad_back = gradient
    for i in layers[::-1]:
      grad_back = i.back_prop(grad_back, learn_rate)

    return loss, acc

# A wrapper function that does only prediction 
def cnn_predict(image, layers):
    out_p = (image/255) - 0.5
    for i in layers:
      out_p = i.forward_prop(out_p)
    return np.argmax(out_p)

# A wrapper function that does only prediction and shows the outputs of the intermediate steps
def cnn_predict_show_inner_workings(image, layers):
    out_p = (image/255) - 0.5
    outputs = []
    for i in layers:
      out_p = i.forward_prop(out_p)
      outputs.append(out_p)

    plots = []
    layer = 1
    for output in outputs[:-1:]:
      fig, axes = plt.subplots(1, output.shape[-1], figsize=(2*output.shape[-1],2))
      axes[0].set_ylabel(f"Layer {layer} output", rotation=90, size='large')
      for i in range(output.shape[-1]):
          ax = axes[i]
          ax.imshow(output[:,:,i], cmap='gray')
      layer += 1
    plt.tight_layout()
    plt.show()
    print("Softmax layer output: ", outputs[-1])
    return np.argmax(outputs[-1])



# Training the model

In [ ]:
history = {"Accuracy":[0], "Average Loss":[0], "TimePerStep":0}
epochs = 5

for epoch in range(epochs):
    print('\nEpoch %d --->' %(epoch + 1))

    # Shuffle the training data
    shuffle_data = np.random.permutation(len(train_images))
    train_images = train_images[shuffle_data]
    train_labels = train_labels[shuffle_data]
    x = train_images[:1000]
    y = train_labels[:1000]
    training_data = zip(x, y)

    # Training the CNN
    loss = history["Average Loss"][epoch]
    num_correct = history['Accuracy'][epoch]
    t1 = time.time()
    for i, (im, label) in enumerate(training_data):
        if i % 100 == 0:
            loss /= 100
            history["TimePerStep"] = (time.time()-t1)/100
            print('Step %d: Average Loss %.3f and Accuracy: %d%% || Time per step = %.3f seconds' %(i+1, loss, num_correct, history["TimePerStep"]))
            loss = num_correct = 0
            t1 = time.time()
        l1, accu = training_cnn(im, label, model)
        loss += l1
        num_correct += accu
    history["Accuracy"].append(num_correct)
    history["Average Loss"].append(loss)


# Graphing the model's training

In [ ]:
plt.plot(range(epochs + 1),history["Accuracy"])
plt.plot(range(epochs + 1),history["Average Loss"])

plt.legend(["Accuracy", "Average Loss"])

plt.xlabel("Epochs")

plt.title("Variation of Training Accuracy and Average Loss with number of epochs")
plt.grid(True)
plt.show()

# Evaluating the model on the Testing data

In [ ]:
t1 = time.time()
num_correct = 0
test_images = test_images[:5000]
test_labels = test_labels[:5000]
test_data = zip(test_images, test_labels)
for i, (im, label) in enumerate(test_data):
    prediction = cnn_predict(im, model)
    if label == prediction:
        num_correct += 1
t2 = time.time()
print(f"Time taken for predicting {len(test_images)} pictures is {t2-t1} s")
print("Testing accuracy = ", num_correct/len(test_images))

# Showing the intermidiate steps involved during prediction

In [ ]:
prediction = cnn_predict_show_inner_workings(test_images[1335], model)
print("Prediction = ", prediction)

In [ ]:
prediction = cnn_predict_show_inner_workings(test_images[1200], model)
print("Prediction = ", prediction)